# 1.4 EntropyMeasure - Classical Shadow
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import ShadowUnveil

experiment_shadow = ShadowUnveil()

/home/nccu/Documents/.homejupyter/qurryplayground/qurrium_docs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_shadow.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_shadow.add(GHZ(i), f"ghz_{i}")
    experiment_shadow.add(make_neel_circuit(i), f"neel_{i}")

experiment_shadow.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7dce04708080>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x7dce046a6e10>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7dce0482e1e0>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7dce04683830>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x7dce30e8f8c0>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7dce75c46fc0>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7dce04646120>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x7dce047c45c0>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7dce046a5eb0>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7dce046c7560>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x7dce046f8080>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.classical_shadow.arguments import ShadowUnveilMeasureArgs

Preparing a configuration list for multiple experiments with following parameters:

```python
class ShadowUnveilMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    times: int
    """The number of random unitary operator. 
    It will denote as `N_U` in the experiment name."""
    measure: Optional[Union[tuple[int, int], int, list[int]]]
    """The measure range."""
    unitary_loc: Optional[Union[tuple[int, int], int, list[int]]]
    """The range of the unitary operator."""
    unitary_loc_not_cover_measure: bool
    """Whether the range of the unitary operator is not cover the measure range."""
    random_unitary_seeds: Optional[dict[int, dict[int, int]]]
    """The seeds for all random unitary operator.
    This argument only takes input as type of `dict[int, dict[int, int]]`.
    The first key is the index for the random unitary operator.
    The second key is the index for the qubit.

    .. code-block:: python
        {
            0: {0: 1234, 1: 5678},
            1: {0: 2345, 1: 6789},
            2: {0: 3456, 1: 7890},
        }

    If you want to generate the seeds for all random unitary operator,
    you can use the function `generate_random_unitary_seeds` 
    in `qurry.qurrium.utils.random_unitary`.

    .. code-block:: python
        from qurry.qurrium.utils.random_unitary import generate_random_unitary_seeds
        random_unitary_seeds = generate_random_unitary_seeds(100, 2)
    """
```


In [4]:
config_list: list[ShadowUnveilMeasureArgs] = [
    {
        "shots": 1024,
        "times": 100,
        "wave": f"{wave_names}_{i}",
        "tags": (wave_names, f"size_{i}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_shadow.multiOutput(
    config_list,
    summoner_name="qurshady",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| Write "qurshady.001", at location "qurshady.001"


| 180/180 100%|██████████| - Loading 180 experiments... - 00:17 < 00:00


| MultiOutput running...


| 180/180 100%|██████████| - Executing completed 'run.001', denoted date: 2025-05-22 14:39:58... - 01:10 < 00:00


| Export multimanager...


| 9/9 - Exporting done - 00:00 < 00:00                           

| No quantity to export.
| Export multi.config.json for cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0



| 179/179 - Exporting experiments... - 00:14 < 00:00
| 180/180 - Loading file infomation... - 00:00 < 00:00

| Exporting file taglist...
| Exporting qurshady.001/qurryinfo.json...
| Exporting qurshady.001/qurryinfo.json done.


'cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0'

You can check the result of `multiOutput` that we just executed by accessing the `.multimanagers`

In [ ]:
experiment_shadow.multimanagers

MultiManagerContainer(num=1, {
  "35981502-1353-452d-b2f4-512c2f9f7f44":
    <MultiManager(name="qurrent.randomized_measure.007", jobstype="local", ..., exps_num=180)>,
})

In [6]:
experiment_shadow.multimanagers[multi_exps1]

<MultiManager(id="cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0",
  name="qurshady.001",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-05-22 14:39:58',},
  exps_num=180)>

### d. Run post-processing at once


In [7]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
    selected_qubits=[0, 1],
)

| 180/180 - Analysis: | taking time of all rho_m: 0.0140 sec:  - 00:02 < 00:00

| "report.001" has been completed.


'cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0'

In [8]:
print("| Available results:")
for k, v in (
    experiment_shadow.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_2') with length 10
| - ('ghz', 'size_2') with length 10
| - ('neel', 'size_2') with length 10
| - ('trivial_paramagnet', 'size_4') with length 10
| - ('ghz', 'size_4') with length 10
| - ('neel', 'size_4') with length 10
| - ('trivial_paramagnet', 'size_6') with length 10
| - ('ghz', 'size_6') with length 10
| - ('neel', 'size_6') with length 10
| - ('trivial_paramagnet', 'size_8') with length 10
| - ('ghz', 'size_8') with length 10
| - ('neel', 'size_8') with length 10
| - ('trivial_paramagnet', 'size_10') with length 10
| - ('ghz', 'size_10') with length 10
| - ('neel', 'size_10') with length 10
| - ('trivial_paramagnet', 'size_12') with length 10
| - ('ghz', 'size_12') with length 10
| - ('neel', 'size_12') with length 10


In [9]:
experiment_shadow.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10")
][:2][0]["expect_rho"]

array([[0.24400879+0.00000000e+00j, 0.20920898+3.67675781e-03j,
        0.23206055-1.90429688e-04j, 0.22662598-8.78906250e-05j],
       [0.20920898-3.67675781e-03j, 0.24664551+0.00000000e+00j,
        0.22337402-6.50390625e-03j, 0.23293945-2.73925781e-03j],
       [0.23206055+1.90429688e-04j, 0.22337402+6.50390625e-03j,
        0.25660645+0.00000000e+00j, 0.21079102-2.03613281e-03j],
       [0.22662598+8.78906250e-05j, 0.23293945+2.73925781e-03j,
        0.21079102+2.03613281e-03j, 0.25273926+0.00000000e+00j]])

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [10]:
expkeys_of_multi_exps1 = list(experiment_shadow.multimanagers[multi_exps1].exps.keys())
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [11]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={
        k: (
            {
                "selected_qubits": [0, 1],
            }
            if idx < 3
            else False
        )
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 180/180 - Analysis: Skipped 9a0bb3ed-5564-4bfb-9bf2-8a998075dfef in cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0. - 00:00 < 00:00

| "first_3.002" has been completed.


'cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0'

In [12]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_shadow.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [13]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={
        k: {
            "selected_qubits": [0, 1],  # selected qubits for the analysis
        }
        for idx, k in enumerate(expkeys_of_multi_exps1)
    },
)

| 180/180 - Analysis: | taking time of all rho_m: 0.0164 sec:  - 00:02 < 00:00


| "all_manual.003" has been completed.
| Export multimanager...


| 9/9 - Exporting done - 00:00 < 00:00                           
| 3/3 - exported quantity complete - 00:00 < 00:00    


| Export multi.config.json for cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0


| 179/179 - Exporting experiments... - 00:22 < 00:00
| 180/180 - Loading file infomation... - 00:00 < 00:00

| Exporting file taglist...
| Exporting qurshady.001/qurryinfo.json...
| Exporting qurshady.001/qurryinfo.json done.


'cc9f67e3-fcc4-4399-bf8e-5316e4f90fa0'

In [14]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_shadow.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [15]:
multi_exps1_reades = experiment_shadow.multiRead(
    save_location=".",
    summoner_name="qurshady.001",
)

| Retrieve qurshady.001...
| at: qurshady.001


| 0/180   0%|          | - Loading 180 experiments ... - 00:00 < ?

| 180/180 100%|██████████| - Loading 180 experiments ... - 00:06 < 00:00


In [16]:
expkeys_of_multi_exps1

['10ccb553-825f-48f6-aa6d-16ed70006f98',
 'eb8d4f2c-4a93-487e-802b-f5ac7507b7ab',
 'a5b67af0-d096-47da-a914-65315095fe40',
 '7c99d7e3-2bcb-4061-ac8b-2014b31965fa',
 'db809f77-7b3e-4612-ba0c-f15f04b8ce64',
 'e99edf03-eb24-4e4d-a737-526d62e9c04f',
 'c96f6947-dfe6-4b82-b094-078155ec4657',
 '827f90f6-c733-4253-a175-31f1fc5c0122',
 '9614bcb7-6689-489f-b7bd-28bf4489c60d',
 '828d4a7c-4b9d-4d62-98b0-c24b4194875a',
 'ad733a47-4dbb-47c8-aca3-3873d711c242',
 '161e3067-0c26-45cd-bbf9-caf6fd7278b5',
 'c43e50f6-2430-4f48-824f-9ce28020d0ae',
 '38984ae8-837d-44dd-b0f9-33bef586a0ad',
 'c8a2deb9-643f-4c3a-95dc-bebf3da6dc19',
 'd835b33f-8d47-43f1-aa55-dc4671a487ea',
 'db0872eb-8e45-4662-b20f-cf84a72ddce5',
 'bb977db8-9e6d-44a4-8766-b11b8ed5f4a1',
 '634bdc2b-728f-4fd0-ab40-d84e3161b666',
 '700e85df-5a31-4ba8-950c-6ae425b20031',
 '793ca250-348a-4154-86a2-7da5c61b4cdc',
 '27143755-a076-4ea2-b1e1-f1662f2f9c02',
 'f8c5dd67-da77-4873-b3b9-08fe23e6c7b3',
 'a3eddb53-fc41-48f1-8f28-5cc437f9518b',
 '711276ca-f479-

---

### Post-Process Availablities and Version Info

In [17]:
from qurry.process import AVAIBILITY_STATESHEET

AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev2
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  